In [1]:
pip install cirq==0.14.0

  Using cached cirq_core-0.14.0-py3-none-any.whl (1.8 MB)


  Attempting uninstall: cirq-core
    Found existing installation: cirq-core 1.1.0
    Uninstalling cirq-core-1.1.0:
      Successfully uninstalled cirq-core-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qsimcirq 0.15.0 requires cirq-core~=1.0, but you have cirq-core 0.14.0 which is incompatible.
openfermion 1.5.1 requires cirq-core>=0.15.0, but you have cirq-core 0.14.0 which is incompatible.
openfermion 1.5.1 requires cirq-google>=0.15.0, but you have cirq-google 0.14.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install qsimcirq

In [ ]:
pip install matplotlib

In [8]:
%load_ext autoreload
%autoreload 2

import cirq
import qsimcirq
import circuits_warehouse as cw
import json
import os
import matplotlib.pyplot as plt
import threading

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
### NOTE only use this if running in google colab
from google.colab import drive
drive.mount('/content/drive')

### MAKE THIS MANUALLY
PS_DIR = '/content/drive/MyDrive/googlecolab-ps-data'
if not os.path.exists(PS_DIR):
    os.makedirs(PS_DIR)

In [3]:
### Use this if you do NOT want to connect to Google drive
PS_DIR = 'ps-data' 
if not os.path.exists(PS_DIR):
    os.makedirs(PS_DIR)

## Heisenberg XXX

In [ ]:
def simulate_heisenbergxxx(err_p):
    T = 30
    N = 15
    nmeas = 30_000
    qubits = cirq.LineQubit.range(N)

    init_bstr = '10'*7 + '1'
    print('Starting at', init_bstr, 'err_p', err_p)
    ideal_timestep = cw.circuit_Heisenberg1D(qubits, .1)
    noisy_timestep = cw.circuit_Heisenberg_noise(qubits, error_rate=err_p, exponent=.1)

    # Ideal simulation
    ideal_timeseries_hist = []
    for t in range(T):
        result = qsimcirq.QSimSimulator().run(
        #result = cirq.Simulator().run(
                            qsimcirq.QSimCircuit([[cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'], 
                            #cirq.Circuit([[cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'], 
                                          cirq.I.on_each(qubits),
                                         [ideal_timestep for _ in range(t)],
                                         cirq.measure(*qubits, key='z')
                                                 ]),
                            repetitions=nmeas
                       ).histogram(key='z')
        ideal_timeseries_hist.append((t,result))

    # save ideal
    with open('{}/heis_ideal_N={}_p={}_shots={}.json'.format(PS_DIR,N,err_p,nmeas),'w') as f:
        f.write(json.dumps(ideal_timeseries_hist, indent=2))
    print('saved ideal simulation, running noise simulation')

    # Noisy simulation
    noisy_timeseries_hist = []
    for t in range(T):
        print('t:',t,'err_p:',err_p)
        result = qsimcirq.QSimSimulator().run(
        #result = cirq.Simulator().run(
                            #cirq.Circuit([[cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'], 
                            qsimcirq.QSimCircuit([[cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'], 
                                          cirq.I.on_each(qubits),
                                         [noisy_timestep for _ in range(t)],   
                                         #[ideal_timestep for _ in range(t)],   
                                         cirq.measure(*qubits, key='z')                                      
                                                 ]),#.with_noise([cirq.bit_flip(p=0.01), cirq.phase_flip(p=0.01)]),
                            repetitions=nmeas
                        ).histogram(key='z')
        noisy_timeseries_hist.append((t,result))        

    # save noisy
    with open('{}/heis_noise_N={}_p={}_shots={}.json'.format(PS_DIR,N,err_p,nmeas),'w') as f:
        f.write(json.dumps(noisy_timeseries_hist, indent=2))

In [ ]:
simulate_heisenbergxxx(0.005)

In [ ]:
simulate_heisenbergxxx(0.01)

In [ ]:
simulate_heisenbergxxx(0.02)
#err_p_list = [0.01, 0.02]

#threads = [threading.Thread(target=simulate_heisenbergxxx, args=(err_p,)) for err_p in err_p_list]

#for t in threads:
#  t.start()

#for t in threads:
#  t.join()

In [ ]:
simulate_heisenbergxxx(0.05)

## T6 QCA

In [ ]:
def simulate_t6qca(err_p):
    T = 30
    N = 15
    nmeas = 30_000
    qubits = cirq.LineQubit.range(N)
    #err_p = .05

    init_bstr = '000000010000000'
    print('Starting at', init_bstr)
    ideal_timestep = cw.circuit_T6QCA(qubits)
    noisy_timestep = cw.circuit_T6QCA_noise(qubits, error_rate=err_p)

    # Ideal simulation
    ideal_timeseries_hist = []
    for t in range(T):
        result = qsimcirq.QSimSimulator().run(
                            cirq.Circuit([cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'], 
                                          cirq.I.on_each(qubits),
                                        [ideal_timestep for _ in range(t)],
                                        cirq.measure(*qubits, key='z')
                                        ),
                            repetitions=nmeas
                      ).histogram(key='z')
        ideal_timeseries_hist.append((t,result))

    # save ideal
    with open('{}/t6qca_ideal_N={}_p={}_shots={}.json'.format(PS_DIR, N,err_p,nmeas),'w') as f:
        f.write(json.dumps(ideal_timeseries_hist, indent=2))
    print('saved ideal simulation, running noise simulation')

    # Noisy simulation
    noisy_timeseries_hist = []
    for t in range(T):
        print('t:',t)
        result = qsimcirq.QSimSimulator().run(
                            cirq.Circuit([cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'], 
                                          cirq.I.on_each(qubits),
                                        [noisy_timestep for _ in range(t)],   
                                        #[ideal_timestep for _ in range(t)],   
                                        cirq.measure(*qubits, key='z')                                      
                                        ),#.with_noise([cirq.bit_flip(p=0.01), cirq.phase_flip(p=0.01)]),
                            repetitions=nmeas
                        ).histogram(key='z')
        noisy_timeseries_hist.append((t,result))        

    # save noisy
    with open('{}/t6qca_noise_N={}_p={}_shots={}.json'.format(PS_DIR,N,err_p,nmeas),'w') as f:
        f.write(json.dumps(noisy_timeseries_hist, indent=2))

In [ ]:
simulate_t6qca(0.005)

In [ ]:
simulate_t6qca(0.01)

In [ ]:
simulate_t6qca(0.02)

In [ ]:
simulate_t6qca(0.05)

### F4 QCA

In [13]:
def simulate_f4qca(err_p):
    T = 30
    N = 15
    nmeas = 10
    qubits = cirq.LineQubit.range(N)
    #err_p = .05

    init_bstr = '000000101000000'
    print('Starting at', init_bstr)

    # Ideal simulation
    ideal_timeseries_hist = []
    for t in range(T):
        #result = qsimcirq.QSimSimulator().run(
        result = cirq.Simulator().run(
                            #qsimcirq.QSimCircuit([[cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'],
                            cirq.Circuit([[cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'], 
                                          cirq.I.on_each(qubits),
                                        [cirq.align_left(cw.circuit_F4QCA(qubits, (ti%2)==0) for ti in range(t))],
                                        cirq.measure(*qubits, key='z')
                            ]),
                            repetitions=nmeas
                      ).histogram(key='z')
        ideal_timeseries_hist.append((t,result))

    # save ideal
    with open('{}/f4qca_ideal_N={}_p={}_shots={}.json'.format(PS_DIR,N,err_p,nmeas),'w') as f:
        f.write(json.dumps(ideal_timeseries_hist, indent=2))
    print('saved ideal simulation, running noise simulation')

    # Noisy simulation
    noisy_timeseries_hist = []
    for t in range(T):
        print('t:',t)
        #result = qsimcirq.QSimSimulator().run(
        result = cirq.Simulator().run(
                            #qsimcirq.QSimCircuit([[cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'],
                            cirq.Circuit([[cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'], 
                                          cirq.I.on_each(qubits),
                                        [cirq.align_left(cw.circuit_F4QCA_noise(qubits, error_rate=err_p, even=(ti%2)==0) for ti in range(t))],   
                                        cirq.measure(*qubits, key='z')                                      
                                        ]),
                            repetitions=nmeas
                        ).histogram(key='z')
        noisy_timeseries_hist.append((t,result))        

    # save noisy
    with open('{}/f4qca_noise_N={}_p={}_shots={}.json'.format(PS_DIR,N,err_p,nmeas),'w') as f:
        f.write(json.dumps(noisy_timeseries_hist, indent=2))

In [14]:
simulate_f4qca(0.005)

Starting at 000000101000000
saved ideal simulation, running noise simulation
t: 0
t: 1
t: 2
t: 3
t: 4
t: 5
t: 6
t: 7
t: 8
t: 9
t: 10
t: 11
t: 12
t: 13
t: 14
t: 15
t: 16
t: 17
t: 18


KeyboardInterrupt: 

In [ ]:
simulate_f4qca(0.01)

In [ ]:
simulate_f4qca(0.02)

In [ ]:
simulate_f4qca(0.05)

### F4 qca with imperfect SPS

In [ ]:
IMPERFECT_DIR = 'imperfect'
if not os.path.exists(PS_DIR+'/'+IMPERFECT_DIR):
    os.makedirs(PS_DIR+'/'+IMPERFECT_DIR)

In [ ]:
T = 30
N = 15
nmeas = 5_000
qubits = cirq.LineQubit.range(N)
err_p = .02

# we got this line from the GatherData.ipynb notebook, which discovers which states are useful here
init_bstr_possible_failures = [
  '001000011111111', '011100000001000', '011110101011111', 
  '011110000010001', '011101000001000', '011000000101010', 
  '011110111110011', '010110111110100', '100010000010000', '101000111011111', '101111010100001', '101000010001000', 
  '111111101111100', '110111111110101', '111111101010111', '111100010111110', '110111111010001', '110101000000100'   
]

start = 0
for ff,init_bstr in enumerate(init_bstr_possible_failures[start:]):
    print('Starting at no', ff+start, init_bstr)
    noisy_timestep = cirq.align_left(cw.circuit_F4QCA_noise(qubits, error_rate=err_p, even=True))

    # Ideal simulation
    ideal_timeseries_hist = []
    for t in range(T):
        result = qsimcirq.QSimSimulator().run(
                            cirq.align_left(cirq.Circuit([cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'], 
                                        cirq.I.on_each(qubits),
                                        [cirq.align_left(cw.circuit_F4QCA(qubits, (ti%2)==0) for ti in range(t))],
                                        cirq.measure(*qubits, key='z')
                                        )),
                            repetitions=nmeas
                      ).histogram(key='z')
        ideal_timeseries_hist.append((t,result))

    # save ideal
    with open('f4qca{}_ideal_N={}_p={}_shots={}.json'.format(init_bstr,N,err_p,nmeas),'w') as f:
        f.write(json.dumps(ideal_timeseries_hist, indent=2))
    print('saved ideal simulation, running noise simulation')

    # Noisy simulation
    noisy_timeseries_hist = []
    for t in range(T):
        print('t:',t)
        result = qsimcirq.QSimSimulator().run(
                            cirq.align_left(cirq.Circuit([cirq.X(qubits[i]) for i,v in enumerate(init_bstr) if v=='1'], 
                                        cirq.I.on_each(qubits),
                                        cirq.align_left([cw.circuit_F4QCA_noise(qubits, error_rate=err_p, even=(ti%2)==0) for ti in range(t)]),
                                        cirq.measure(*qubits, key='z')                                      
                                        )),
                            repetitions=nmeas
                        ).histogram(key='z')
        noisy_timeseries_hist.append((t,result))        

    # save noisy
    with open('{}/{}/f4qca{}_noise_N={}_p={}_shots={}.json'.format(PS_DIR,IMPERFECT_DIR,init_bstr,N,err_p,nmeas),'w') as f:
        f.write(json.dumps(noisy_timeseries_hist, indent=2))